# Group Wildfire Data Spatially and Temporally

In [1]:
import pandas as pd 
import numpy as np
import os
import csv

# Function to read a CSV file while keeping rows even if the first column is empty
def read_csv_file(file_path, date_column_name):
    # Read the file starting from the second row, and keep rows even if the first column is empty
    data = pd.read_csv(file_path, skiprows=1, keep_default_na=False)

    # Strip any whitespace from the column names
    data.columns = data.columns.str.strip()

    return data

# Function to calculate a fire vector for a specific week based on geographical splits
def get_fire_vector_for_week(kelowna_stations_df, burn_data_df, num_long_splits, num_lat_splits):
    # Initialize a result array with zeros to store fire sizes
    res = np.zeros(num_long_splits * num_lat_splits)

    # Determine the maximum and minimum longitude and latitude from the station data
    longitude_max = kelowna_stations_df['Longitude'].max()
    longitude_min = kelowna_stations_df['Longitude'].min()
    latitude_max = kelowna_stations_df['Latitude'].max()
    latitude_min = kelowna_stations_df['Latitude'].min()

    # Calculate the total distance covered by the stations in both longitude and latitude
    longitude_distance = longitude_max - longitude_min
    latitude_distance = latitude_max - latitude_min
    
    # Determine the size of each split in longitude and latitude
    long_split_distance = longitude_distance / num_long_splits
    lat_split_distance = latitude_distance / num_lat_splits
    
    # Initialize tracking variables for longitude and latitude
    long_track = longitude_min
    lat_track = latitude_min
    
    # Iterate through each split in latitude
    for i in range(num_lat_splits):
        lat_track = latitude_min + i * lat_split_distance  # Update the latitude tracker
        # Iterate through each split in longitude within the current latitude split
        for j in range(num_long_splits):
            long_track = longitude_min + j * long_split_distance  # Update the longitude tracker
            
            # Create a mask to filter burn data within the current latitude and longitude split
            latitude_mask_burn = (burn_data_df['LATITUDE'] < lat_track + lat_split_distance) & (burn_data_df['LATITUDE'] >= lat_track)
            longitude_mask_burn = (burn_data_df['LONGITUDE'] < long_track + long_split_distance) & (burn_data_df['LONGITUDE'] >= long_track)
            
            # Sum the fire sizes within the current split and store it in the result array
            single_burn = burn_data_df[latitude_mask_burn & longitude_mask_burn]['FIRE_SIZE_HA'].sum(axis=0)
            res[j + i * num_long_splits] = single_burn
            
        # Reset the longitude tracker for the next latitude split
        long_track = longitude_min
    
    return res

# Function to generate a DataFrame representing the fire vector for a specific week
def get_week_fire_df(file_path, kelowna_stations_df, num_long_splits, num_lat_splits):
    # Read the burn data from the CSV file
    burn_data_df = pd.read_csv(file_path)
    
    # Calculate the fire vector for the week based on the burn data and station data
    week_fire = get_fire_vector_for_week(kelowna_stations_df, burn_data_df, num_long_splits, num_lat_splits)
    
    # Print the fire vector for debugging purposes
    print(week_fire)
    
    # Convert the fire vector into a DataFrame and return it
    week_fire_df = pd.DataFrame(week_fire)
    return week_fire_df

In [2]:
kelowna_stations_df = pd.read_csv('../data/processing/weather-stations/kelowna/kelowna_stations.csv')
df = get_week_fire_df("../data/processing/satellite-burn/2017/week_1.csv", kelowna_stations_df, 12, 12)
df

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


,0
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0
...,...
139,0.0
140,0.0
141,0.0
142,0.0
